# Feature Detection, Matching, Segmentation

<center><img src="figs/03_main.png" width=500px alt="default"/></center>


Consider the twopairs of images shown in Figure 7.2. For the first pair,we may wish to align the twoimages so that theycan be seamlessly stitched into acomposite mosaic (Section 8.2). For the second pair,we may wish to establish adense set of correspondences so that a3D model can be constructed or an in-between viewcan be generated (Chapter 12). In either case, what kinds of features should you detect and then match to establish such an alignment or set of correspondences?
